# TO DO!

- figure out how to deal with the fact that data is not great!

In [11]:
require(ggplot2)
source("00_functions.R")


 loading required packages



## read in data

In [12]:
trees_df   = read.csv("../Results/trees_genus_matrix.csv", row.names=1)
mammals_df = read.csv("../Results/mammals_matrix.csv", row.names=1)
beetles_df = read.csv("../Results/beetles_matrix.csv", row.names=1)

In [13]:
pca.rslt_trees   = do_pca(trees_df, scale = F, plot = F)
pca.rslt_mammals = do_pca(mammals_df, scale = F, plot = F)
pca.rslt_beetles = do_pca(beetles_df, scale = F, plot = F)

# cat("Explained Varience Trees:")
# pca.rslt_trees@exp.var
# cat("Explained Varience Mammals:")
# pca.rslt_mammals@exp.var
# cat("Explained Varience Beetles:")
# pca.rslt_beetles@exp.var

In [ ]:
hvs_rslts_trees   = hvs_rslts(pca.rslt_trees@axis, axis = c("PC1", "PC2", "PC3"), "seq")
hvs_rslts_mammals = hvs_rslts(pca.rslt_mammals@axis, axis = c("PC1", "PC2", "PC3"), "seq")
hvs_rslts_beetles = hvs_rslts(pca.rslt_beetles@axis, axis = c("PC1", "PC2", "PC3"), "seq")

# cat("\n\n")
# cat("Trees: ",   sum(is.na(hvs.rslts_trees@rslts$centroid_PC1))/nrow(hvs.rslts_trees@rslts), "\n")
# cat("Mammals: ", sum(is.na(hvs.rslts_mammals@rslts$centroid_PC1))/nrow(hvs.rslts_mammals@rslts), "\n")
# cat("Beetles: ", sum(is.na(hvs.rslts_beetles@rslts$centroid_PC1))/nrow(hvs.rslts_beetles@rslts))


Building Hypervolume  36  of  36 :  Tower_c44
Comparing Hypervolume  27  of  27 :  Tower_c3-c44

Building Hypervolume  36  of  36 :  E100-2_c2017
Comparing Hypervolume  30  of  30 :  E100-2_c2016-c2017


In [ ]:
plot_hvs(hvs_rslts_trees)

In [ ]:
hvs_rslts_beetles@compare[order(hvs_rslts_beetles@compare$overlap), ]

In [ ]:
beetles = data.frame(group = "beetles", overlap = hvs_rslts_beetles@compare$overlap)
trees   = data.frame(group = "trees", overlap = hvs_rslts_trees@compare$overlap)
mammals = data.frame(group = "mammals", overlap = hvs_rslts_mammals@compare$overlap)

whisker = rbind(beetles, trees, mammals)
whisker = whisker[complete.cases(whisker), ]

In [ ]:
ggplot(data = whisker, aes(x = group, y = overlap, color = group)) + geom_boxplot() + geom_jitter()

## model???!!!

In [9]:
attributes <- read.csv("../Results/trees_sorted.csv")
a = as.data.frame(attributes %>% group_by(plot_c) %>% summarise(stem_C = sum(stem_C)))

hvs_rslts_trees@rslts   = merge(hvs_rslts_trees@rslts, a, by = "plot_c", all.x = T)
# need to figure out compare - maybe do change in AGB too?
# hvs_rslts_trees@compare = merge(hvs_rslts_trees@compare, a, by = "plot_c", all.x = T)


Call:
lm(formula = centroid_PC1 ~ stem_C * census, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.6518 -0.4126 -0.2036  0.3720  1.0688 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)  
(Intercept)      7.510e-01  3.522e-01   2.132   0.0444 *
stem_C          -6.318e-06  2.470e-06  -2.558   0.0179 *
censusc2         1.625e-01  5.039e-01   0.323   0.7501  
censusc3         1.030e-01  5.111e-01   0.202   0.8422  
censusc4        -8.221e-02  7.768e-01  -0.106   0.9167  
stem_C:censusc2 -5.197e-07  3.458e-06  -0.150   0.8819  
stem_C:censusc3  5.730e-07  3.589e-06   0.160   0.8746  
stem_C:censusc4  5.119e-07  1.121e-05   0.046   0.9640  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6109 on 22 degrees of freedom
  (6 observations deleted due to missingness)
Multiple R-squared:  0.4853,	Adjusted R-squared:  0.3216 
F-statistic: 2.964 on 7 and 22 DF,  p-value: 0.02395
